# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import os, sys
from importlib import reload
from config import data_config
from utils.args_parser import add_data_args

# Reload arg_parser because Jupyter notebook does not reload it automatically
reload(data_config)

# Add arguments to parser
parser = ArgumentParser()
parser = add_data_args(parser)

# Print default dataset parameters
args = parser.parse_args([])
print("\n=====================")
print("Default dataset parameters:")
for arg in vars(args):
    print(arg, '=', getattr(args, arg))
print("\n=====================")




Default dataset parameters:
ode_system = pendulum
t_max = 20
step_size = 0.01
n_sample = 10
x_init_pts = [[-3.14159265  3.14159265]
 [-8.          8.        ]]
ctr_func = gaussian
datafile_path = data/pendulum_ctr_gaussian_N_10_T20.npy



In [4]:
# Run database_generator.py with arguments
print("\n=====================")
print("Running database_generator.py with arguments:")
%run database_generator.py --n_sample 50 --ctr_func "gaussian" --t_max 10 --datafile_path "data/pendulum_ctr_grf_N_50_T10.npy"



Running database_generator.py with arguments:


/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in subtract
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in sin
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
Generating 50 data ...: 100%|██████████| 50/50 [00:08<00:00,  5.83it/s]

Data saved in data/pendulum_ctr_grf_N_50_T10.npy .


In [1]:
%run train.py --datafile_path "data/pendulum_ctr_grf_N_5000_T10.npy" 

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using all available 1 GPUs.
Shapes for LSTM-MIONet training input=(18000, 999, 1), x_n=(18000, 1), x_next=(18000, 1)
Shapes for LSTM-MIONet training input=(2000, 999, 1), x_n=(2000, 1), x_next=(2000, 1)
Data memory size: 68 MB
Data memory size: 7 MB
LSTM_MIONet_Static(
  (layernorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (layernorm_x_n): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (layernorm_delta_t): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (lstm): LSTM(150, 10, num_layers=2, batch_first=True)
  (cell_mlp): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=150, bias=True)
  )
  (input_mlp): Sequential(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=150, bias=True)
  )
  (delta_t_mlp): Sequential(
    (0): Lin

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Index is supposed to be an empty tensor or a vector